# DATA TRANSFORMATION - 19_consolidacion
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf
import adherence_functions as af
import warnings
warnings.filterwarnings('ignore')

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Varios

In [2]:
calculos = gf.diccionario_agg_functions()
dcc = gf.diccionario_llaves()

In [3]:
# execute initial file
files = 'C:/Users/monic/documentos/ds4a/ds4a_team80/1_data_transformation/1_consolidacion_mensual/'
file_0 = files + '0_initial.py'

exec(open(file_0).read())

## 2. Cálculo de Variables por Módulo

### 2.1. Funciones

In [59]:
# Para el cálculo de variables 
def cal_variables( base_features, base_ad_12, idx, nmeses, calculos, cc, dc_n  ):
    
    # Cruza con adherencia
    base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
    
    # Limita el número de meses
    base_1 = base_cruce[base_cruce['d_meses'] <= nmeses]
    
    dc_n[ cc ] = {}
    
    # Realiza agrupaciones de acuerdo al cálculo
    for calculo in calculos[ cc ]:
        
        if calculo == 'sum' :
            variables = calculos[ cc ][calculo]
            print(variables)
            print(nmeses)
            base_12 = base_1.groupby( idx )[variables].sum().reset_index()
            
            sufi = '_' + calculo + '_' + str(nmeses)
            base_12.columns = [ s + sufi for s in base_12.columns]
            base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
            dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            
        elif calculo == 'avg' :
            variables = calculos[ cc ][calculo]
            print(variables)
            print(nmeses)
            base_12 = base_1.groupby( idx )[variables].mean().reset_index()
            
            sufi = '_' + calculo + '_' + str(nmeses)
            base_12.columns = [ s + sufi for s in base_12.columns]
            base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
            dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            
    return dc_n


### 2.2. Varios

In [5]:
base_ini = gf.base_ids_mensual( ruta_archivos )
base_ad_12, base_ad = af.base_features_adeherencia0( ruta_archivos, dcc, ids )

In [62]:
base_ad_12[ ( base_ad_12['id']==500547) & ( base_ad_12['year_obs']==2016) & ( base_ad_12['month_obs']==7) ]

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses
0,500547,2016,7,1,1,2016,1,6
1,500547,2016,7,1,1,2016,2,5
2,500547,2016,7,1,1,2016,3,4
3,500547,2016,7,1,1,2016,4,3
4,500547,2016,7,1,1,2016,5,2
5,500547,2016,7,1,1,2016,6,1


In [7]:
base_ad[ ( base_ad['id']==500547) & ( base_ad['year']==2016) & ( base_ad['month']==7) ]

,id,year,month,adeherencia_0,key
0,500547,2016,7,1,1


In [8]:
print(base_ini.shape)
base_ini.head()

(52320, 3)


,id,year,month
0,500547,2016,1
1,500547,2016,2
2,500547,2016,3
3,500547,2016,4
4,500547,2016,5


In [60]:
idx = ['id', 'year_obs', 'month_obs']
nmeses = [6,12]
dc_n = {}

for cc in calculos :
    
    # Ejecuta el modulo
    m = calculos[cc]['mod']
    file_i = files + dcc[ m ]['prefi'] + '.py'
    exec(open(file_i).read())
    exec("base_i = base_final_" + dcc[m]['prefi'] )
    print( "Ejecutando modulo " + cc )
    print( base_i.shape )
    
    # Quita los nulls
    cols_i = base_i.columns

    base_i['nuls'] = base_i.isnull().sum(axis = 1)
    todo_null = base_i.shape[1] - 4    #id, month, year, nuls
    base_features = base_i[base_i['nuls'] < todo_null ]
    base_features.reset_index( inplace = True )
    base_features = base_features[cols_i]
    print(base_features.shape)
    
    # Calcula las variables
    for mn in nmeses : 
        dc_n = cal_variables( base_features, base_ad_12, idx, mn, calculos, cc, dc_n  )


Ejecutando modulo far
(52320, 14)
(248, 14)
['far_rea_asma', 'far_rea_total', 'far_evo_aun', 'far_cau_def', 'far_cau_otra', 'far_gra_leve', 'far_gra_mod']
6
['far_rea_asma', 'far_rea_total', 'far_evo_aun', 'far_cau_def', 'far_cau_otra', 'far_gra_leve', 'far_gra_mod']
12
Ejecutando modulo hos
(52320, 11)
(576, 11)
['hos_num', 'hos_num_j', 'hos_uci', 'hos_uci_j', 'hos_uce', 'hos_uce_j', 'hos_est', 'hos_est_j']
6
['hos_num', 'hos_num_j', 'hos_uci', 'hos_uci_j', 'hos_uce', 'hos_uce_j', 'hos_est', 'hos_est_j']
12
Ejecutando modulo med
(52320, 9)
(19162, 9)
['med_num_dis', 'med_flag_j', 'med_flag_otra']
6
['med_num_doses_j', 'med_num_doses_otra']
6
['med_num_dis', 'med_flag_j', 'med_flag_otra']
12
['med_num_doses_j', 'med_num_doses_otra']
12
Ejecutando modulo urg
(52320, 7)
(2059, 7)
['urg_urg', 'urg_total', 'urg_j_urg', 'urg_j_total']
6
['urg_urg', 'urg_total', 'urg_j_urg', 'urg_j_total']
12
Ejecutando modulo vac
(52320, 4)
(215, 4)
['vac_cant']
6
['vac_cant']
12
Ejecutando modulo cal
(5232

In [58]:
for ind in dc_n['med']:
    print(ind)

sum_12
avg_12


In [65]:
data = dc_n['med']['sum_12']

In [66]:
data[(data['id'] == 500547) & (data['year_obs'] == 2019)]

,id,year_obs,month_obs,med_num_dis_sum_12,med_flag_j_sum_12,med_flag_otra_sum_12
0,500547,2019,3,40.0,1.0,9.0
1,500547,2019,10,36.0,1.0,11.0
2,500547,2019,12,40.0,1.0,11.0


In [67]:
base_ad[ ( base_ad['id']==500547) & ( base_ad['year']==2019)  ]

,id,year,month,adeherencia_0,key
1,500547,2019,3,1,1
2,500547,2019,10,1,1
3,500547,2019,12,1,1


In [69]:
base_ad_12[ ( base_ad_12['id']==500547) & ( base_ad_12['year_obs']==2019) & ( base_ad_12['month_obs']==3)  ]

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses
60,500547,2019,3,1,1,2016,1,38
61,500547,2019,3,1,1,2016,2,37
62,500547,2019,3,1,1,2016,3,36
63,500547,2019,3,1,1,2016,4,35
64,500547,2019,3,1,1,2016,5,34
65,500547,2019,3,1,1,2016,6,33
66,500547,2019,3,1,1,2016,7,32
67,500547,2019,3,1,1,2016,8,31
68,500547,2019,3,1,1,2016,9,30
69,500547,2019,3,1,1,2016,10,29


In [37]:
base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
base_cruce.head()

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses,year,month,med_flag_j,med_num_dis_otra,med_num_doses_j,med_num_doses_otra,med_num_dis,med_flag_otra
0,500547,2019,3,1,1,2016,12,27,2016,12,0.0,5.0,0.0,1670.0,5.0,1.0
1,500547,2019,10,1,1,2016,12,34,2016,12,0.0,5.0,0.0,1670.0,5.0,1.0
2,500547,2019,12,1,1,2016,12,36,2016,12,0.0,5.0,0.0,1670.0,5.0,1.0
3,500547,2020,1,1,1,2016,12,37,2016,12,0.0,5.0,0.0,1670.0,5.0,1.0
4,500547,2020,3,1,1,2016,12,39,2016,12,0.0,5.0,0.0,1670.0,5.0,1.0


In [48]:
base_12 = base_cruce[base_cruce['d_meses'] <= 12]
base_12.head()

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses,year,month,med_flag_j,med_num_dis_otra,med_num_doses_j,med_num_doses_otra,med_num_dis,med_flag_otra
48,500547,2019,3,1,1,2018,5,10,2018,5,0.0,5.0,0.0,570.0,5.0,1.0
54,500547,2019,3,1,1,2018,6,9,2018,6,0.0,5.0,0.0,755.0,5.0,1.0
60,500547,2019,3,1,1,2018,7,8,2018,7,0.0,5.0,0.0,1111.0,5.0,1.0
66,500547,2019,3,1,1,2018,8,7,2018,8,0.0,5.0,0.0,1296.0,5.0,1.0
72,500547,2019,3,1,1,2018,10,5,2018,10,0.0,4.0,0.0,951.0,4.0,1.0


In [29]:
dc_n[ cc ] = {}

In [43]:
variables = calculos[ cc ]['avg']
variables
#base_12 = base_12.groupby( idx )[variables].sum().reset_index()

['med_num_doses_j', 'med_num_doses_otra']

In [49]:
base_12 = base_12.groupby( idx )[variables].mean().reset_index()
base_12.head()

,id,year_obs,month_obs,med_num_doses_j,med_num_doses_otra
0,500547,2019,3,0.111111,982.333333
1,500547,2019,10,0.090909,658.090909
2,500547,2019,12,0.090909,639.818182
3,500547,2020,1,0.000000,643.454545
4,500547,2020,3,0.090909,672.545455


In [50]:
sufi = '_' + 'avg' + '_' + str(12)
base_12.columns = [ s + sufi for s in base_12.columns]
base_12.head()

,id_avg_12,year_obs_avg_12,month_obs_avg_12,med_num_doses_j_avg_12,med_num_doses_otra_avg_12
0,500547,2019,3,0.111111,982.333333
1,500547,2019,10,0.090909,658.090909
2,500547,2019,12,0.090909,639.818182
3,500547,2020,1,0.000000,643.454545
4,500547,2020,3,0.090909,672.545455


In [51]:
base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
#dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
base_12.head()      

,id,year_obs,month_obs,med_num_doses_j_avg_12,med_num_doses_otra_avg_12
0,500547,2019,3,0.111111,982.333333
1,500547,2019,10,0.090909,658.090909
2,500547,2019,12,0.090909,639.818182
3,500547,2020,1,0.000000,643.454545
4,500547,2020,3,0.090909,672.545455


In [ ]:
base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
    
# Limita el número de meses
base_12 = base_cruce[base_cruce['d_meses'] <= nmeses]

dc_n[ cc ] = {}

# Realiza agrupaciones de acuerdo al cálculo
for calculo in calculos[ cc ]:
    
    if calculo == 'sum' :
        variables = calculos[ cc ][calculo]
        base_12 = base_12.groupby( idx )[variables].sum().reset_index()
        
        sufi = '_' + calculo + '_' + str(nmeses)
        base_12.columns = [ s + sufi for s in base_12.columns]
        base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
        dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
        
    elif calculo == 'avg' :
        variables = calculos[ cc ][calculo]
        base_12 = base_12.groupby( idx )[variables].mean().reset_index()
        
        sufi = '_' + calculo + '_' + str(nmeses)
        base_12.columns = [ s + sufi for s in base_12.columns]
        base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
        dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            

In [15]:
base_features.head()

,id,year,month,cal_ent,cal_psi,cal_rel,cal_fis
0,502989,2017,1,63.0,69.0,50.0,63.0
1,524006,2016,3,100.0,94.0,94.0,100.0
2,524006,2016,8,100.0,94.0,94.0,100.0
3,524006,2016,12,88.0,81.0,69.0,69.0
4,524006,2017,8,75.0,56.0,31.0,50.0


In [17]:
base_ad_12.head()

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses
0,500547,2016,7,1,1,2016,1,6
1,500547,2016,7,1,1,2016,2,5
2,500547,2016,7,1,1,2016,3,4
3,500547,2016,7,1,1,2016,4,3
4,500547,2016,7,1,1,2016,5,2


In [63]:
dc_n['cal']['avg_12']

,id,year_obs,month_obs,cal_ent_avg_12,cal_psi_avg_12,cal_rel_avg_12,cal_fis_avg_12
0,502989,2017,3,63.0,69.0,50.0,63.0
1,502989,2017,7,63.0,69.0,50.0,63.0
2,502989,2017,8,63.0,69.0,50.0,63.0
3,524006,2016,6,100.0,94.0,94.0,100.0
4,524006,2016,10,100.0,94.0,94.0,100.0
...,...,...,...,...,...,...,...
395,1962088,2017,1,63.0,69.0,69.0,31.0
396,1991750,2017,5,81.0,69.0,69.0,63.0
397,1991750,2017,10,81.0,69.0,69.0,63.0
398,1997065,2017,5,81.0,94.0,69.0,63.0


In [18]:
base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
print(base_cruce.shape)
base_cruce.head()

(1403, 14)


,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses,year,month,cal_ent,cal_psi,cal_rel,cal_fis
0,502989,2017,3,1,1,2017,1,2,2017,1,63.0,69.0,50.0,63.0
1,502989,2017,7,1,1,2017,1,6,2017,1,63.0,69.0,50.0,63.0
2,502989,2017,8,1,1,2017,1,7,2017,1,63.0,69.0,50.0,63.0
3,502989,2018,10,0,1,2017,1,21,2017,1,63.0,69.0,50.0,63.0
4,502989,2019,2,1,1,2017,1,25,2017,1,63.0,69.0,50.0,63.0


In [37]:

print(base_12.shape)

(603, 14)


In [38]:
for calculo in calculos[ cc ]:
    if calculo == 'sum' :
        variables = calculos[ cc ][calculo]
        base_12 = base_12.groupby( idx )[variables].sum().reset_index()
    elif calculo == 'avg' :
        variables = calculos[ cc ][calculo]
        base_12 = base_12.groupby( idx )[variables].mean().reset_index()

In [39]:
base_12.head()

,id,year_obs,month_obs,cal_ent,cal_psi,cal_rel,cal_fis
0,502989,2017,3,63.0,69.0,50.0,63.0
1,502989,2017,7,63.0,69.0,50.0,63.0
2,502989,2017,8,63.0,69.0,50.0,63.0
3,524006,2016,6,100.0,94.0,94.0,100.0
4,524006,2016,10,100.0,94.0,94.0,100.0


In [40]:
base_12.head()

,id,year_obs,month_obs,cal_ent_var_12,cal_psi_var_12,cal_rel_var_12,cal_fis_var_12
0,502989,2017,3,63.0,69.0,50.0,63.0
1,502989,2017,7,63.0,69.0,50.0,63.0
2,502989,2017,8,63.0,69.0,50.0,63.0
3,524006,2016,6,100.0,94.0,94.0,100.0
4,524006,2016,10,100.0,94.0,94.0,100.0


In [42]:
base_12.shape

(400, 7)

In [34]:
exec("base_i = base_ini.merge( base_final_" + dcc[m]['prefi'] + ", how='left')")

In [37]:
base_final_far.shape

(52320, 14)

In [35]:
base_i.head()

,id,year,month,far_rea_asma,far_rea_total,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod
0,500547,2016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print(base_i.shape)

cols_i = base_i.columns

(52320, 14)


In [30]:
base_i.head()

,id,year,month,far_rea_asma,far_rea_total,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod,nuls
0,500547,2016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
1,500547,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
2,500547,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
3,500547,2016,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
4,500547,2016,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11


In [31]:
todo_null = base_i.shape[1] - 4
todo_null

11

In [38]:
base_i.groupby('nuls')['id'].count().reset_index(name='cuenta').sort_values('nuls')

,nuls,cuenta
0,0,248
1,11,52072


In [36]:
print(base_i.shape)

cols_i = base_i.columns

base_i['nuls'] = base_i.isnull().sum(axis = 1)
todo_null = base_i.shape[1] - 4    #id, month, year, nuls
base_features = base_i[base_i['nuls'] < todo_null ]
base_features.reset_index( inplace = True )
base_features = base_features[cols_i]
print(base_features.shape)

base_features.head()

(52320, 14)
(248, 14)


,id,year,month,far_rea_asma,far_rea_total,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod
0,525540,2019,12,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,530049,2016,9,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,535255,2016,11,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,535255,2019,10,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,539423,2016,9,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
for m in mods :
    print(m)
    if m != 'Adherencia' :
        file_i = files + dcc[m]['prefi'] + '.py'
        exec(open(file_i).read())
    
        exec("base_consolidada = base_consolidada.merge( base_final_" + dcc[m]['prefi'] + ", how='left')")

Calidad de vida relacioada en salud


NameError: name 'base_consolidada' is not defined

In [ ]:
base_consolidada.columns

In [ ]:
base_consolidada.shape

In [ ]:
base_consolidada.head()

In [ ]:
cols_base_consolidada = base_consolidada.columns

In [ ]:
base_consolidada['nuls'] = base_consolidada.isnull().sum(axis = 1)

In [ ]:
base_consolidada.groupby('nuls')['id'].count().reset_index(name='cuenta').sort_values('nuls')

In [ ]:
todo_null = base_consolidada.shape[1] - 4    #id, month, year, nuls
base_features = base_consolidada[base_consolidada['nuls'] < todo_null ]
base_features.reset_index( inplace = True )
base_features = base_features[cols_base_consolidada]
base_features.shape

In [ ]:
base_features.head()

# Consolidación Variables
## Cruce con Adherencia

In [ ]:
base_ad.head()

In [ ]:
base_ad.describe()

In [ ]:
base_ad = af.base_features_adeherencia0( ruta_archivos, dcc, ids )
base_cruce = base_ad.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )

In [ ]:
base_cruce.shape

## Definiciones varias

In [ ]:
idx = ['id', 'year_x', 'month_x']

In [ ]:
calculos = {
    'sum'  : {'variables' : ['far_rea_asma', 'far_rea_total','far_evo_aun','far_cau_def', 'far_cau_otra','far_gra_leve','far_gra_mod',
                            'hos_num','hos_num_j','hos_uci','hos_uci_j','hos_uce','hos_uce_j','hos_est','hos_est_j',
                            'med_num_dis','med_flag_j','med_flag_otra',
                            'urg_urg', 'urg_total','urg_j_urg', 'urg_j_total','vac_cant'] },
    'avg'  : {'variables' : ['cal_ent','cal_psi', 'cal_rel', 'cal_fis','med_num_doses_j', 'med_num_doses_otra'] },
    'flag' : {'variables' : ['far_rea_asma', 'far_rea_total','far_rea_asma', 'far_rea_total','far_rea_asma', 'far_rea_total','far_evo_con','far_des_si','far_des_no','far_mis_si',
                                    'med_flag_j','med_flag_otra','vac_cant']},
    'var'  : {'variables' : ['cal_ent','cal_psi', 'cal_rel', 'cal_fis',
                             'med_num_doses_j', 'med_num_doses_otra']}
           }

# 12 meses

In [ ]:
nmeses = 12
calculo = 'sum'
base_12 = base_cruce[base_cruce['d_meses'] <= nmeses]
base_12.shape

variables_insumo = calculos[calculo]['variables']
variables_insumo

base_12_sum = base_12.groupby(idx)[variables_insumo].sum().reset_index()

base_12_sum.sort_values('hos_num', ascending = False).head()

In [ ]:
base_12_sum.head()

In [ ]:
base_12_sum.sort_values('hos_num', ascending = False).head()